In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf

from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import mlp4, cnn_lenet
from sklearn.impute import SimpleImputer

In [2]:
afq_dataset = AFQDataset(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants.csv",
    dwi_metrics=["dki_fa", "dki_md"],
    target_cols=["age"],
    index_col="subject_id",
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1867
(1867, 4800)
(1867,)


In [5]:
# Here we impute missing bundles.
# We impute using the entire dataset, which permits data leakage between the train and test set.
# THIS IS BAD AND SHOULDN'T BE DONE IN PRODUCTION
# But we do it here to move straight to model training
# When we are more comfortabel with the models, we should come back
# here and train the imputer only on the training set.
imputer = SimpleImputer(strategy="median")
afq_dataset.X = imputer.fit_transform(afq_dataset.X)

In [6]:
# Separate into training, test, and validation sets
dataset_size = len(afq_dataset.subjects)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)

full_dataset = afq_dataset.as_tensorflow_dataset()
full_dataset = full_dataset.shuffle(buffer_size=dataset_size, seed=0)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [7]:
# Batch the datasets for ingestion during model training/evaluation
batch_size = 128
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [8]:
# First let's check out the ``cnn_lenet`` from the tf_models module
model = cnn_lenet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)

pooling layers: 4
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 48)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 6)            870       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 50, 6)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 16)            304       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 25, 26)            1274      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 12, 26)

In [9]:
# Compile the model and fit it using training data
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model.fit(train_dataset, epochs=100)

Epoch 1/100
11/11 [==============================] - 1s 22ms/step - loss: 66.2018 - mean_squared_error: 66.2018
Epoch 2/100
11/11 [==============================] - 0s 18ms/step - loss: 28.4324 - mean_squared_error: 28.4324
Epoch 3/100
11/11 [==============================] - 0s 19ms/step - loss: 23.4638 - mean_squared_error: 23.4638
Epoch 4/100
11/11 [==============================] - 0s 18ms/step - loss: 22.2581 - mean_squared_error: 22.2581
Epoch 5/100
11/11 [==============================] - 0s 19ms/step - loss: 21.6447 - mean_squared_error: 21.6447
Epoch 6/100
11/11 [==============================] - 0s 19ms/step - loss: 18.9925 - mean_squared_error: 18.9925
Epoch 7/100
11/11 [==============================] - 0s 19ms/step - loss: 19.5974 - mean_squared_error: 19.5974
Epoch 8/100
11/11 [==============================] - 0s 21ms/step - loss: 19.7697 - mean_squared_error: 19.7697
Epoch 9/100
11/11 [==============================] - 0s 19ms/step - loss: 18.1667 - mean_squared_error: 

In [10]:
# Evaluate the model on the validation set
model.evaluate(val_dataset)

3/3 [==============================] - 0s 7ms/step - loss: 6.3917 - mean_squared_error: 6.3917


[6.391738414764404, 6.391738414764404]

We see that this model achieves a mean squared error of ~6.4, or an RMSE of ~2.5 years. This is probably an overly optimistic estimate because we imputed using the entire dataset.